In [2]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

2024-04-03 14:25:36.867971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 14:25:36.868073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 14:25:37.005026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPU(s), {len(logical_gpus)} Logical GPU(s)')
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found, using CPU instead.")

1 Physical GPU(s), 1 Logical GPU(s)


In [4]:
# Load the dataset
df = pd.read_csv('/kaggle/input/custom3/sequence_target_map_ritutoshniwal.csv')

# Preprocess data
# Convert the sequences of moves into strings (if they aren't already)
df['input_sequence'] = df['input_sequence'].astype(str)
df['target_move'] = df['target_move'].astype(str)

In [5]:
# Tokenize the input sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['input_sequence'])
sequences = tokenizer.texts_to_sequences(df['input_sequence'])
max_sequence_len = max([len(x) for x in sequences])
print(max_sequence_len)
total_words = len(tokenizer.word_index) + 1

178


In [6]:
# Prepare input data for the LSTM
X = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

# Encode target moves
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['target_move'])
y = to_categorical(integer_encoded)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=5, min_lr=0.0001)

# Stop training when a monitored metric has stopped improving
early_stopping = EarlyStopping(monitor='accuracy', patience=10, verbose=1)

# Build the GRU model
model = Sequential()
model.add(Embedding(total_words, 128, input_length=max_sequence_len))
model.add(GRU(400))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print model summary
print(model.summary())

# Train the model
model.fit(
    X_train,
    y_train,
    epochs=20,
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 178, 128)          168192    
                                                                 
 gru (GRU)                   (None, 400)               636000    
                                                                 
 dropout (Dropout)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 1164)              466764    
                                                                 
Total params: 1270956 (4.85 MB)
Trainable params: 1270956 (4.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/20


I0000 00:00:1712154357.048920      87 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1354/1354 [==============================] - 48s 33ms/step - loss: 5.2873 - accuracy: 0.1085 - val_loss: 4.8223 - val_accuracy: 0.1492 - lr: 0.0010
Epoch 2/20
1354/1354 [==============================] - 29s 22ms/step - loss: 4.5093 - accuracy: 0.1808 - val_loss: 4.4725 - val_accuracy: 0.1878 - lr: 0.0010
Epoch 3/20
1354/1354 [==============================] - 28s 21ms/step - loss: 4.0458 - accuracy: 0.2216 - val_loss: 4.3447 - val_accuracy: 0.2027 - lr: 0.0010
Epoch 4/20
1354/1354 [==============================] - 27s 20ms/step - loss: 3.6184 - accuracy: 0.2620 - val_loss: 4.3596 - val_accuracy: 0.2120 - lr: 0.0010
Epoch 5/20
1354/1354 [==============================] - 28s 20ms/step - loss: 3.1614 - accuracy: 0.3193 - val_loss: 4.4496 - val_accuracy: 0.2023 - lr: 0.0010
Epoch 6/20
1354/1354 [==============================] - 27s 20ms/step - loss: 2.7094 - accuracy: 0.3898 - val_loss: 4.5756 - val_accuracy: 0.2095 - lr: 0.0010
Epoch 7/20
1354/1354 [==============================] - 2

In [8]:
# Save the model
model_json = model.to_json()
with open("model_arch.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")

# Save the tokenizer and label encoder using pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Encode a new sequence to predict the next move
def predict_next_move(input_sequence):
    sequence = tokenizer.texts_to_sequences([input_sequence])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_len, padding='pre')
    prediction = model.predict(padded_sequence)
    predicted_move_index = np.argmax(prediction)
    predicted_move = label_encoder.inverse_transform([predicted_move_index])
    return predicted_move[0]

In [9]:
# Example usage
print(predict_next_move("d4 d5 Nf3 Nf6 e3 e6 Bd3 Bd6 Nbd2 O-O O-O Nbd7 Qe2 Qe7 e4 Nxe4 Nxe4 Qxe4 a6"))

1/1 [==============================] - 1s 670ms/step
Ba4
